In [ ]:
import torch
from utils.dataset import ScenarioGraphDataset
from torch_geometric.loader import DataLoader
from utils.data_aug_model import Generator, Discriminator, train_gan, evaluate_generator
from utils.dataset_utils import NODE_TYPE_MAP
import torch.optim as optim

import os
os.environ["OMP_NUM_THREADS"] = '1'

# ------------------ 主程序示例 ------------------
if __name__ == "__main__":
    # 假设已有场景及数据路径
    scene_datasets = {
        "secondary_road": ["dataset/driving-scene-graph/secondary-road"],
        "ebike": ['dataset/driving-scene-graph/ebike'],
        "main_secondary": ['dataset/driving-scene-graph/main-secondary'],
        "motor": [
            "dataset/driving-scene-graph/secondary-road",
            'dataset/driving-scene-graph/main-secondary'
        ],
        "total": [
            "dataset/driving-scene-graph/secondary-road",
            'dataset/driving-scene-graph/main-secondary',
            'dataset/driving-scene-graph/ebike'
        ]
    }
    # 数据集参数
    window_size = 30  # 窗口步长
    step_size = 1  # 假设的步长值
    node_feature_dim = 19 + len(NODE_TYPE_MAP)  # 节点特征维度 # 假设改为10
    num_classes = 3  # 标签类别数量

    # 训练参数
    hidden_dim = 64
    num_epochs = 100
    batch_size = 48
    num_workers = 8  # 使用 4 个进程加载数据
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 逐场景训练
    for scene_name, root_dirs in scene_datasets.items():
        print(f"\n=== 开始训练场景: {scene_name} ===")

        # 定义缓存路径，针对每个场景使用单独的缓存文件
        cache_path = f"dataset/cache/{scene_name}_{window_size}_{step_size}_dataset_pre_cache.pkl" 
        generator_model_path = f"model/data_aug/{scene_name}_{window_size}_{step_size}_generator_model.pth"
        checkpoint_path = f"model/checkpoint/{scene_name}_generator_model_checkpoint.pth" # 定义检查点路径

        dataset = ScenarioGraphDataset(root_dirs, window_size, step_size, device, cache_path)
        # 假设数据集的 80% 用于训练，20% 用于验证
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

        # 在初始化 DataLoader
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

        # 初始化生成器和判别器, 并添加 Dropout
        generator = Generator(node_feature_dim, hidden_dim, window_size, num_classes, dropout_rate=0.5).to(device)
        discriminator = Discriminator(node_feature_dim, hidden_dim, window_size, dropout_rate=0.5).to(device)
    
        # 初始化优化器
        g_optimizer = optim.AdamW(generator.parameters(), lr=1e-3, weight_decay=1e-4)
        d_optimizer = optim.AdamW(discriminator.parameters(), lr=1e-3, weight_decay=1e-4)
    
        # 训练 GAN
        train_gan(train_dataloader, generator, discriminator, g_optimizer, d_optimizer, num_epochs, device, window_size, val_dataloader, checkpoint_path, scene_name,num_classes=num_classes, generator_model_path=generator_model_path)

        # 保存模型
        # torch.save(generator.state_dict(), generator_model_path) # 已在早停中保存
        # torch.save(discriminator.state_dict(), f"{scene_name}_discriminator.pth")
        print(f"场景 {scene_name} 训练完成\n")


=== 开始训练场景: secondary_road ===
从缓存文件加载数据: dataset/cache/secondary_road_30_1_dataset_pre_cache.pkl


Epoch 1/100:   0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader
from stnet.stnet import SpatioTemporalModel
from utils.utils import ModelConfig, FocalLoss, train_model
from utils.dataset import AugmentedScenarioGraphDataset
from utils.dataset import ScenarioGraphDataset
from utils.dataset_utils import NODE_TYPE_MAP, EDGE_TYPE_MAP

if __name__ == '__main__':
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.set_float32_matmul_precision('high')

    # 滑动窗口配置
    window_size = 30
    step_size = 1

    # 初始化配置
    config = ModelConfig(
        num_layers=3,
        num_features=12 + len(NODE_TYPE_MAP),
        hidden_dim=16,
        num_relations=8,
        edge_dim=8,
        num_epochs=200,
        num_classes=3,
        window_size=window_size,
        step_size=step_size
    )

    working_dir = "./"
    model_dir = f"{working_dir}/model"
    dataset_dir = "./dataset"

    # 数据集配置
    scene_datasets = {
        "main_secondary": ['/main-secondary'],
        "secondary_road": ["/secondary-road"],
        "motor": [
            "/secondary-road",
            '/main-secondary'
        ],
        # "ebike": ['/ebike'],
        # "total": [
        #     "/secondary-road",
        #     '/main-secondary',
        #     '/ebike'
        # ]
    }

    # 训练循环
    for scene_name, data_dirs in scene_datasets.items():
        print(f"\n=== Training Scene: {scene_name} ===")
        
        data_dirs = [f"{dataset_dir}\driving-scene-graph{d}" for d in data_dirs]
        
        # 数据集加载
        cache_path = f"{dataset_dir}/cache/{scene_name}_{window_size}_{step_size}_dataset_pre_cache.pkl"
        generator_model_path = f"{working_dir}/model/data_aug/{scene_name}_{window_size}_{step_size}_generator_model.pth"

        # 加载数据集
        dataset = ScenarioGraphDataset(data_dirs, window_size, step_size, device, cache_path)
        weights = dataset.compute_class_weights()
        
        # 划分训练集和验证集
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, 
                                                                    [train_size, val_size], 
                                                                    generator=torch.Generator().manual_seed(42))

        # 创建数据加载器
        train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

        # 模型初始化
        model = SpatioTemporalModel(config).to(device)
        optimizer = optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-4)
        # criterion = FocalLoss(alpha=0.25, gamma=2)
        criterion = nn.CrossEntropyLoss(weight=weights.to(device))
        print(f"class weights: {weights}")

        # 开始训练
        train_model(
            model=model,
            train_loader=train_loader,
            val_loader=train_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            config=config,
            checkpoint_dir=f"model/checkpoint/",
            bestmodel_dir=f"model/data_aug/",
            scene_name=scene_name,
            patience=15
        )

In [ ]:
import os
import torch
import wandb
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader
from stnet.stnet import SpatioTemporalModel
from utils.utils import ModelConfig, FocalLoss, train_model
from utils.dataset import AugmentedScenarioGraphDataset
from utils.dataset_utils import NODE_TYPE_MAP, EDGE_TYPE_MAP

if __name__ == '__main__':
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.set_float32_matmul_precision('high')

    # 滑动窗口配置
    window_size = 30
    step_size = 1

    # 初始化配置
    config = ModelConfig(
        num_layers=3,
        num_features=12 + len(NODE_TYPE_MAP),
        hidden_dim=16,
        num_relations=8,
        edge_dim=8,
        num_epochs=200,
        num_classes=3,
        window_size=window_size,
        step_size=step_size
    )

    working_dir = "./"
    model_dir = f"{working_dir}/model"
    dataset_dir = "./dataset"

    # 数据集配置
    scene_datasets = {
        "motor": [
            "/secondary-road",
            '/main-secondary'
        ]
    }
    num_layer = 2
    num_hidden = 16

    # 训练循环
    for scene_name, data_dirs in scene_datasets.items():
        print(f"\n=== Training Scene: {scene_name} ===")
        
        # 初始化 wandb
        wandb.init(config={})  # 自动使用配置参数
        config = wandb.config  # 获取超参数配置

        # 数据集路径配置
        data_dirs = [f"{dataset_dir}/driving-scene-graph{d}" for d in data_dirs]

        # 数据集加载
        cache_path = f"{dataset_dir}/cache/{scene_name}_{config.window_size}_{config.step_size}_dataset_aug_cache.pkl"
        generator_model_path = f"{working_dir}/model/data_aug/{scene_name}_{config.window_size}_{config.step_size}_{num_layer}_{num_hidden}_generator_model.pth"

        # 加载数据集
        dataset = AugmentedScenarioGraphDataset(
            root_dirs=data_dirs,
            window_size=config.window_size,
            step_size=config.step_size,
            generator_model_path=generator_model_path,
            node_feature_dim=config.num_features,
            device=device,
            cache_path=cache_path,
            num_classes=config.num_classes
        )

        # 划分训练集和验证集
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, 
                                                                    [train_size, val_size], 
                                                                    generator=torch.Generator().manual_seed(0))

        # 创建数据加载器
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

        # 模型初始化
        model = SpatioTemporalModel(config).to(device)
        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
        criterion = nn.CrossEntropyLoss()

        # 开始训练
        train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            config=config,
            checkpoint_dir=f"{model_dir}/checkpoint",
            bestmodel_dir=f"{model_dir}/bestmodel",
            scene_name=scene_name,
            patience=15
        )


=== Training Scene: secondary_road ===
从缓存文件加载数据: ./dataset/cache/secondary_road_30_1_dataset_aug_cache.pkl


Epoch 1/200: 100%|██████████| 16/16 [04:22<00:00, 16.41s/it, loss=0.0162]


Epoch 1 | Train Loss: 0.0156 | Val Acc: 0.3333 | Val Prec: 0.1113 | Val Recall: 0.3333 | Val F1: 0.1669


Epoch 2/200: 100%|██████████| 16/16 [03:39<00:00, 13.73s/it, loss=0.0151]


Epoch 2 | Train Loss: 0.0150 | Val Acc: 0.3338 | Val Prec: 0.2843 | Val Recall: 0.3338 | Val F1: 0.1749


...

Epoch 3 | Train Loss: 0.0150 | Val Acc: 0.3333 | Val Prec: 0.1176 | Val Recall: 0.3333 | Val F1: 0.1738


Epoch 4/200:  62%|██████▎   | 10/16 [02:51<01:42, 17.12s/it, loss=0.0149]


Interrupt received, saving checkpoint...
Checkpoint saved at epoch 3
Best validation F1: 0.1749


NameError: name 'exit' is not defined

: 

In [ ]:
import os
import torch
import wandb
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader
from stnet.stnet import SpatioTemporalModel
from utils.utils import ModelConfig, FocalLoss, train_model
from utils.dataset import AugmentedScenarioGraphDataset
from utils.dataset_utils import NODE_TYPE_MAP, EDGE_TYPE_MAP


def train(config=None):
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.set_float32_matmul_precision('high')
    with wandb.init(config=None, mode="offline"):
        configW = wandb.config
        print(configW)  

        # 初始化配置
        config = ModelConfig(
            num_layers=configW.num_layers,
            num_features=12 + len(NODE_TYPE_MAP),
            hidden_dim=configW.hidden_dim,
            num_relations=8,
            edge_dim=8,
            num_epochs=5,
            num_classes=configW.num_classes,
            window_size=configW.window_size,
            step_size=configW.step_size,
            learning_rate=configW.learning_rate,
            weight_decay=configW.weight_decay,
            graph_num_head=configW.graph_num_head,
            pool_ratio=configW.pool_ratio,
            num_seed_points=configW.num_seed_points,
            graph_dropout=configW.graph_dropout,
            lstm_hidden_dim=configW.lstm_hidden_dim,
            lstm_bidirectional=configW.lstm_bidirectional,
            lstm_num_layers=configW.lstm_num_layers,
            fc_dropout=configW.fc_dropout,
            batch_size=configW.batch_size,
        )

        working_dir = "./"
        model_dir = f"{working_dir}/model"
        dataset_dir = "./dataset"

        # 数据集配置
        scene_datasets = {
            "motor": [
                "/secondary-road",
                '/main-secondary'
            ]
        }
        scene_name = "motor"
        data_dirs = scene_datasets[scene_name]
        
        print(f"\n=== Training Scene: {scene_name} ===")
        
        # 数据集路径配置
        data_dirs = [f"{dataset_dir}/driving-scene-graph{d}" for d in data_dirs]

        # 数据集加载
        cache_path = f"{dataset_dir}/cache/{scene_name}_{config.window_size}_{config.step_size}_dataset_aug_cache.pkl"
        generator_model_path = f"{working_dir}/model/data_aug/{scene_name}_{30}_{1}_{2}_{16}_generator_model.pth"

        # 加载数据集
        dataset = AugmentedScenarioGraphDataset(
            root_dirs=data_dirs,
            window_size=config.window_size,
            step_size=config.step_size,
            generator_model_path=generator_model_path,
            node_feature_dim=config.num_features,
            device=device,
            cache_path=cache_path,
            num_classes=config.num_classes
        )

        # 划分训练集和验证集
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, 
                                                                    [train_size, val_size], 
                                                                    generator=torch.Generator().manual_seed(0))

        # 创建数据加载器
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

        # 模型初始化
        model = SpatioTemporalModel(config).to(device)
        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
        criterion = nn.CrossEntropyLoss()

        # 开始训练
        train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            device=device,
            config=config,
            checkpoint_dir=f"{model_dir}/checkpoint",
            bestmodel_dir=f"{model_dir}/bestmodel",
            scene_name=scene_name,
            patience=15
        )

if __name__ == '__main__':
    import yaml
    with open("config/sweep.yaml", "r", encoding="utf-8") as file:
        config_dict = yaml.safe_load(file)
    sweep_id = wandb.sweep(config_dict, project="pytorch-sweeps-demo")
    wandb.agent(sweep_id, train, count=1)

Create sweep with ID: ov1u8zna
Sweep URL: https://wandb.ai/1422909005/pytorch-sweeps-demo/sweeps/ov1u8zna


wandb: Agent Starting Run: qjshsl7i with config:
wandb: 	batch_size: 64
wandb: 	fc_dropout: 0.1
wandb: 	graph_dropout: 0.8
wandb: 	graph_num_head: 1
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.01
wandb: 	lstm_bidirectional: False
wandb: 	lstm_hidden_dim: 32
wandb: 	lstm_num_layers: 2
wandb: 	num_classes: 3
wandb: 	num_layers: 4
wandb: 	num_seed_points: 4
wandb: 	pool_ratio: 0.7
wandb: 	step_size: 1
wandb: 	weight_decay: 0.01
wandb: 	window_size: 10
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


{'batch_size': 64, 'fc_dropout': 0.1, 'graph_dropout': 0.8, 'graph_num_head': 1, 'hidden_dim': 64, 'learning_rate': 0.01, 'lstm_bidirectional': False, 'lstm_hidden_dim': 32, 'lstm_num_layers': 2, 'num_classes': 3, 'num_layers': 4, 'num_seed_points': 4, 'pool_ratio': 0.7, 'step_size': 1, 'weight_decay': 0.01, 'window_size': 10}

=== Training Scene: motor ===
从缓存文件加载数据: ./dataset/cache/motor_10_1_dataset_aug_cache.pkl


Epoch 1/5:   2%|▏         | 2/84 [00:36<22:47, 16.68s/it, loss=1.1516]